**Vide link:**

https://youtu.be/Y_wiJANVcL0

**Github link:**

https://github.com/rezash121/Sentiment-Analysis-Demo-Custom-Fine-Tuned-Model-vs.-Llama-3


In [ ]:
# Import the necessary libraries
import pandas as pd
import kagglehub
import os

# Download the latest version of the dataset from Kaggle
dataset_dir = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
print("Path to dataset files:", dataset_dir)

# List the files in the downloaded directory to find the CSV file
files = os.listdir(dataset_dir)
print("Files in the dataset directory:", files)

csv_file = os.path.join(dataset_dir, "IMDB Dataset.csv")

# Load the dataset using Pandas
df = pd.read_csv(csv_file)

# Verify the dataset by printing the first few rows and its info
print("\nFirst five rows of the dataset:")
print(df.head())

print("\nDataset information:")
print(df.info())


Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1
Files in the dataset directory: ['IMDB Dataset.csv']

First five rows of the dataset:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None


In [ ]:
from sklearn.model_selection import train_test_split


# Step 1: Encode the sentiment column: positive -> 1, negative -> 0.
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Step 2: Retain only the review and label columns.
df = df[['review', 'label']]

# Verify changes: print first few rows and info
print("Dataset after preprocessing (first five rows):")
print(df.head())
print("\nDataset info:")
print(df.info())

# Step 3: Split the data into training, validation, and testing sets.

train_val_df, test_df = train_test_split(df, test_size=0.20, random_state=42, stratify=df['label'])

train_df, val_df = train_test_split(train_val_df, test_size=0.10, random_state=42, stratify=train_val_df['label'])

# Print the sizes of the splits
print("\nDataset split sizes:")
print("Training set:", train_df.shape)
print("Validation set:", val_df.shape)
print("Testing set:", test_df.shape)


Dataset after preprocessing (first five rows):
                                              review  label
0  One of the other reviewers has mentioned that ...      1
1  A wonderful little production. <br /><br />The...      1
2  I thought this was a wonderful way to spend ti...      1
3  Basically there's a family where a little boy ...      0
4  Petter Mattei's "Love in the Time of Money" is...      1

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB
None

Dataset split sizes:
Training set: (36000, 2)
Validation set: (4000, 2)
Testing set: (10000, 2)


In [ ]:
from transformers import AutoTokenizer

# Step 1: Select the pre-trained Hugging Face transformer model for fine-tuning.
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Define tokenization parameters:
tokenization_params = dict(truncation=True, padding='max_length', max_length=256)

# Tokenize the datasets:
# Convert the 'review' column from each pandas DataFrame into a list and tokenize.
train_encodings = tokenizer(train_df['review'].tolist(), **tokenization_params)
val_encodings = tokenizer(val_df['review'].tolist(), **tokenization_params)
test_encodings = tokenizer(test_df['review'].tolist(), **tokenization_params)

# Display a sample tokenized output from the training set.
print("Sample tokenized output (from Training set):")
sample_tokens = {key: train_encodings[key][0] for key in train_encodings}
print(sample_tokens)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Sample tokenized output (from Training set):
{'input_ids': [101, 1996, 3185, 4627, 2125, 2004, 2057, 2156, 1037, 8333, 1997, 1037, 4121, 14734, 2067, 1999, 1996, 2382, 1005, 1055, 1999, 2637, 1012, 2059, 1037, 2460, 2466, 2003, 3491, 2055, 1037, 17109, 1011, 2559, 7500, 12005, 3900, 2040, 2081, 1037, 3066, 2007, 16795, 1010, 2000, 2131, 2204, 11203, 1012, 12005, 3900, 5086, 2402, 2273, 2000, 2147, 1999, 2010, 3871, 1010, 2730, 2068, 1010, 1998, 2109, 2004, 12665, 24375, 2015, 1012, 2002, 2036, 7349, 1996, 2598, 2007, 2037, 2668, 1012, 2070, 2051, 2044, 1016, 10558, 2272, 2000, 3942, 2032, 1012, 2028, 1997, 2068, 4152, 2915, 2011, 12005, 3900, 1010, 2178, 2028, 8563, 1996, 7500, 2370, 1012, 1012, 1012, 2044, 2008, 1010, 1996, 2556, 2154, 2003, 3491, 1010, 1998, 2070, 3124, 2315, 5977, 2003, 2409, 2008, 2002, 2038, 2019, 2214, 3888, 2187, 2004, 12839, 1012, 2002, 7288, 2000, 2175, 2045, 2007, 2070, 2814, 2000, 2156, 2054, 1005, 1055, 2039, 1012, 2210, 2106, 5977, 2113, 2008, 1996, 2279, 

In [ ]:
import torch
import numpy as np
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Step 1: Load the pre-trained model for sequence classification.
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Step 2: Define a custom dataset class to work with the Trainer.
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset instances for training, validation, and testing.
train_dataset = IMDbDataset(train_encodings, train_df["label"].tolist())
val_dataset   = IMDbDataset(val_encodings, val_df["label"].tolist())
test_dataset  = IMDbDataset(test_encodings, test_df["label"].tolist())

# Step 3: Define the metric computation function.
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Step 4: Set up training arguments.
training_args = TrainingArguments(
    output_dir="./results",            # Output directory for model predictions and checkpoints.
    num_train_epochs=2,                # Total number of training epochs.
    per_device_train_batch_size=16,    # Batch size for training (adjust to 16 or 32 as desired).
    per_device_eval_batch_size=16,     # Batch size for evaluation.
    evaluation_strategy="epoch",       # Evaluation is done at the end of each epoch.
    learning_rate=5e-5,                # Learning rate.
    logging_strategy="epoch",          # Log metrics at the end of each epoch.
    save_strategy="epoch",             # Save checkpoints at the end of each epoch.
    load_best_model_at_end=True,       # Load the best model when finished training.
    metric_for_best_model="f1",        # Use F1 score to evaluate the best model.
    logging_dir='./logs',              # Directory for storing logs.
)

# Step 5: Instantiate the Trainer.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Step 6: Fine-tune the model.
trainer.train()

# Optional: Evaluate the model on the validation set.
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rza121 (rzash121) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.282900,0.208570,0.918000,0.920523,0.915000,0.917753
2,0.138600,0.268623,0.920750,0.912702,0.930500,0.921515


Evaluation results: {'eval_loss': 0.2686232924461365, 'eval_accuracy': 0.92075, 'eval_precision': 0.9127023050514959, 'eval_recall': 0.9305, 'eval_f1': 0.9215152265412231, 'eval_runtime': 29.5001, 'eval_samples_per_second': 135.593, 'eval_steps_per_second': 8.475, 'epoch': 2.0}


In [ ]:
# Save the fine-tuned model and tokenizer locally.
model_save_path = "./imdb-finetuned-model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model and tokenizer saved locally at {model_save_path}")

# Log in to the Hugging Face Hub.
from huggingface_hub import login # Import login function

login(token="##############") # Use login function with token argument


# Upload (push) the model and tokenizer to the Hugging Face Hub.
repo_name = "rza121/imdb-finetuned-model"
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
!pip install fastapi uvicorn pyngrok


In [ ]:
!killall uvicorn || echo "No uvicorn process found"

In [ ]:
# Import required modules
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import threading

# Apply nest_asyncio to allow nested event loops (required for Colab)
nest_asyncio.apply()

# Kill existing ngrok tunnels to avoid exceeding the limit
ngrok.kill()

# Optionally kill any server processes using port 8003 (uncomment if necessary)
!fuser -k 8003/tcp

# Set your ngrok authtoken (replace with your actual authtoken)
ngrok.set_auth_token("##############")

# Step 2: Define the FastAPI app and endpoints
app = FastAPI()

# Define the request body structure using Pydantic.
class AnalyzeRequest(BaseModel):
    text: str
    model: str

# Define the response structure.
class AnalyzeResponse(BaseModel):
    sentiment: str
    confidence: float

@app.post("/analyze/", response_model=AnalyzeResponse)
async def analyze(request: AnalyzeRequest):
    # Check for empty text.
    if not request.text:
        raise HTTPException(status_code=400, detail="Input text cannot be empty.")

    # Dummy sentiment analysis logic.
    if request.model.lower() == "custom":
        if "good" in request.text.lower():
            sentiment = "positive"
            confidence = 0.95
        else:
            sentiment = "negative"
            confidence = 0.90
    elif request.model.lower() == "llama":
        if "happy" in request.text.lower():
            sentiment = "positive"
            confidence = 0.93
        else:
            sentiment = "negative"
            confidence = 0.88
    else:
        raise HTTPException(status_code=400, detail="Invalid model specified. Choose 'custom' or 'llama'.")

    return AnalyzeResponse(sentiment=sentiment, confidence=confidence)

# Step 3: Run the FastAPI app using Uvicorn in a background thread

# Define the port number (using 8003)
port = 8003

# Start ngrok tunnel on port 8003
tunnel = ngrok.connect(port)
public_url = tunnel.public_url
print("ngrok tunnel created! Public URL:", public_url)

# Function to run the Uvicorn server
def run():
    uvicorn.run(app, host="0.0.0.0", port=port)

# Start the server in a new thread.
thread = threading.Thread(target=run)
thread.start()


In [ ]:
import requests

# Build the endpoint URL
endpoint_url = public_url + "/analyze/"
print("Endpoint URL:", endpoint_url)

# Define the payload for the POST request.
payload = {"text": "This movie was really good!", "model": "custom"}

# Send the POST request and print the JSON response.
response = requests.post(endpoint_url, json=payload)
print("Response:", response.json())


Endpoint URL: https://4076-34-125-214-192.ngrok-free.app/analyze/
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK
Response: {'sentiment': 'positive', 'confidence': 0.95}


In [ ]:
!pip install groq
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import requests
import os

##############################
# 1. Load the Fine-Tuned Model from Hugging Face
##############################

# Replace with your actual repository name on Hugging Face Hub
model_repo = "rza121/imdb-finetuned-model"

# Load the fine-tuned model and tokenizer
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(model_repo)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(model_repo)

print("Fine-tuned model and tokenizer loaded from Hugging Face!")

##############################
# 2. Access the Llama 3 Model via Groq Cloud API using the provided snippet
##############################

from groq import Groq

# Create a Groq client using your API key from an environment variable.
client = Groq(api_key="###############")

# Request a chat completion from the Llama 3 model.
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say Hello world!",
        }
    ],
    model="llama-3.3-70b-versatile",
)

# Print the Llama 3 model response.
print("Llama 3 response:")
print(chat_completion.choices[0].message.content)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fine-tuned model and tokenizer loaded from Hugging Face!
Llama 3 response:
Hello world!


In [ ]:
# Import required modules
import os
import torch
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import threading
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from groq import Groq
import re

# Apply nest_asyncio to allow nested event loops (required for Colab)
nest_asyncio.apply()

# Kill existing ngrok tunnels to avoid exceeding the limit
ngrok.kill()

# Optionally kill any processes using port 8003
!fuser -k 8003/tcp

# Set your ngrok authtoken (replace with your actual authtoken)
ngrok.set_auth_token("####################")

groq_client = Groq(api_key="####################")
app = FastAPI()

# Define the request body structure using Pydantic.
class AnalyzeRequest(BaseModel):
    text: str
    model: str

# Define the response structure.
class AnalyzeResponse(BaseModel):
    sentiment: str
    confidence: float

@app.post("/analyze/", response_model=AnalyzeResponse)
async def analyze(request: AnalyzeRequest):
    if not request.text:
        raise HTTPException(status_code=400, detail="Input text cannot be empty.")

    # Option 1: Use the fine-tuned Hugging Face model ("custom")
    if request.model.lower() == "custom":
        # Tokenize input text with truncation, padding and a max length of 256
        inputs = fine_tuned_tokenizer(
            request.text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=256,
        )
        with torch.no_grad():
            outputs = fine_tuned_model(**inputs)
        logits = outputs.logits
        # Compute probabilities and select the highest scoring class
        probs = torch.softmax(logits, dim=-1)
        confidence_tensor, pred_class_tensor = torch.max(probs, dim=1)
        confidence = confidence_tensor.item()
        pred_class = pred_class_tensor.item()
        # Assume label mapping: 1 -> positive, 0 -> negative.
        sentiment = "positive" if pred_class == 1 else "negative"
        return AnalyzeResponse(sentiment=sentiment, confidence=confidence)

      # Option 2: Use the Groq Cloud API to access the Llama 3 model ("llama")
    elif request.model.lower() == "llama":
        chat_completion = groq_client.chat.completions.create(
            messages=[
                {"role": "user", "content": request.text},
            ],
            model="llama-3.3-70b-versatile",
        )
        # Get the raw response content from Llama 3
        llama_response = chat_completion.choices[0].message.content
        print("Llama Response (raw):", llama_response)  # Debug output

        # Determine sentiment from the response
        if "positive" in llama_response.lower():
            sentiment = "positive"
        else:
            sentiment = "negative"

        # Initialize confidence variable to avoid unbound errors
        confidence = None

        # First, look for a pattern like "confidence: 0.92" (case insensitive)
        match = re.search(r"score[:\s]*([\d]+\.[\d]+)", llama_response, re.IGNORECASE)
        if match:
            try:
                confidence = float(match.group(1))
            except ValueError:
                confidence = 0.9
        else:
            # As a fallback, search for any floating point number in the response
            match = re.search(r"(\d+\.\d+)", llama_response)
            if match:
                try:
                    confidence = float(match.group(1))
                except ValueError:
                    confidence = 0.9
            else:
                confidence = 0.9  # Default value if no number is found

        return AnalyzeResponse(sentiment=sentiment, confidence=confidence)

    else:
        raise HTTPException(status_code=400, detail="Invalid model specified. Choose 'custom' or 'llama'.")

##############################
# Start the API with Uvicorn and Expose via ngrok
##############################

# Define the port number (using 8003)
port = 8003

# Start an ngrok tunnel on the selected port
tunnel = ngrok.connect(port)
public_url = tunnel.public_url  # e.g., "http://xxxx.ngrok.io"
print("ngrok tunnel created! Public URL:", public_url)

# Function to run the Uvicorn server
def run():
    uvicorn.run(app, host="0.0.0.0", port=port)

# Start the server in a background thread.
thread = threading.Thread(target=run)
thread.start()

ngrok tunnel created! Public URL: https://8515-34-125-214-192.ngrok-free.app


INFO:     Started server process [68250]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8003 (Press CTRL+C to quit)


In [ ]:
import requests
import json
import os

# ------------------------------------------------------------------------------
# Instructions for Testing with Postman:
# ------------------------------------------------------------------------------
# 1. Open Postman and create a new POST request.
# 2. Set the request URL to:
#       {public_url}/analyze/
#    (Replace {public_url} with the actual ngrok URL printed by your server.)
# 3. Set the header "Content-Type" to "application/json".
# 4. For testing the "custom" model, use this JSON in the body:
#       {
#           "text": "This movie was really good!",
#           "model": "custom"
#       }
#    For testing the "llama" model, use:
#       {
#           "text": "Explain the importance of fast language models",
#           "model": "llama"
#       }
# 5. Send the request and verify the response.
#
# ------------------------------------------------------------------------------
# Testing the /analyze/ endpoint using Python requests:
# ------------------------------------------------------------------------------
endpoint_url = public_url + "/analyze/"
print("Endpoint URL:", endpoint_url)

# Test with the "custom" model:
payload_custom = {"text": "This movie was really good!", "model": "custom"}
response_custom = requests.post(endpoint_url, json=payload_custom)
print("\nPython requests response for custom model:")
print(response_custom.json())

# Test with the "llama" model:
payload_llama = {"text": "This movie was really good!", "model": "llama"}
response_llama = requests.post(endpoint_url, json=payload_llama)
print("\nPython requests response for llama model:")
print(response_llama.json())

# ------------------------------------------------------------------------------
# Testing the /analyze/ endpoint using curl commands:
# ------------------------------------------------------------------------------
print("\nTesting with curl (outputs below):")

# Note: In Google Colab, you can use shell commands with !. The variable endpoint_url
# is substituted by placing it in curly braces inside a string.
os.environ['ENDPOINT_URL'] = endpoint_url

print("\nTesting with curl (custom model):")
!curl -X POST "$ENDPOINT_URL" -H "Content-Type: application/json" -d '{"text": "This movie was really good!", "model": "custom"}'

print("\nTesting with curl (llama model):")
!curl -X POST "$ENDPOINT_URL" -H "Content-Type: application/json" -d '{"text": "This movie was really good!", "model": "llama"}'


Endpoint URL: https://8515-34-125-214-192.ngrok-free.app/analyze/
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK

Python requests response for custom model:
{'sentiment': 'positive', 'confidence': 0.9951962828636169}
Llama Response (raw): I'm glad you enjoyed the movie. What was it about that you liked? Was it the storyline, the characters, the acting, or something else? Would you like to discuss it or get recommendations for similar movies?
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK

Python requests response for llama model:
{'sentiment': 'negative', 'confidence': 0.9}

Testing with curl (outputs below):

Testing with curl (custom model):
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK
{"sentiment":"positive","confidence":0.9951962828636169}
Testing with curl (llama model):
Llama Response (raw): That's great to hear! I'd love to know more about the movie you enjoyed. Can you tell me which one it was and what you liked about it? Was it a 

In [ ]:
import requests
import json
import os

# ------------------------------------------------------------------------------
# Instructions for Testing with Postman:
# ------------------------------------------------------------------------------
# 1. Open Postman and create a new POST request.
# 2. Set the request URL to:
#       {public_url}/analyze/
#    (Replace {public_url} with the actual ngrok URL printed by your server.)
# 3. Set the header "Content-Type" to "application/json".
# 4. For testing the "custom" model, use this JSON in the body:
#       {
#           "text": "This movie was really good!",
#           "model": "custom"
#       }
#    For testing the "llama" model, use:
#       {
#           "text": "Explain the importance of fast language models",
#           "model": "llama"
#       }
# 5. Send the request and verify the response.
#
# ------------------------------------------------------------------------------
# Testing the /analyze/ endpoint using Python requests:
# ------------------------------------------------------------------------------
endpoint_url = public_url + "/analyze/"
print("Endpoint URL:", endpoint_url)

# Test with the "custom" model:
payload_custom = {"text": "This movie was really good!", "model": "custom"}
response_custom = requests.post(endpoint_url, json=payload_custom)
print("\nPython requests response for custom model:")
print(response_custom.json())

# Test with the "llama" model:
payload_llama = {"text": "Classify the sentiment of this text as positive or negative: "+"This movie was really good!", "model": "llama"}
response_llama = requests.post(endpoint_url, json=payload_llama)
print("\nPython requests response for llama model:")
print(response_llama.json())

# ------------------------------------------------------------------------------
# Testing the /analyze/ endpoint using curl commands:
# ------------------------------------------------------------------------------
print("\nTesting with curl (outputs below):")

# Note: In Google Colab, you can use shell commands with !. The variable endpoint_url
# is substituted by placing it in curly braces inside a string.

os.environ['ENDPOINT_URL'] = endpoint_url

print("\nTesting with curl (custom model):")
!curl -X POST "$ENDPOINT_URL" -H "Content-Type: application/json" -d '{"text": "This movie was really good!", "model": "custom"}'

print("\nTesting with curl (llama model):")
!curl -X POST "$ENDPOINT_URL" -H "Content-Type: application/json" -d '{"text": "Classify the sentiment of this text as positive or negative: This movie was really good!", "model": "llama"}'


Endpoint URL: https://8515-34-125-214-192.ngrok-free.app/analyze/
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK

Python requests response for custom model:
{'sentiment': 'positive', 'confidence': 0.9951962828636169}
Llama Response (raw): The sentiment of this text is positive. The phrase "really good" indicates a favorable opinion of the movie.
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK

Python requests response for llama model:
{'sentiment': 'positive', 'confidence': 0.9}

Testing with curl (outputs below):

Testing with curl (custom model):
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK
{"sentiment":"positive","confidence":0.9951962828636169}
Testing with curl (llama model):
Llama Response (raw): The sentiment of this text is positive. The word "good" has a positive connotation, and the emphasis added by "really" strengthens the positive sentiment. Overall, the text expresses a favorable opinion of the movie.
INFO:     34.125.214.192:

In [ ]:

os.environ['ENDPOINT_URL'] = endpoint_url

print("\nTesting with curl (custom model):")
!curl -X POST "$ENDPOINT_URL" -H "Content-Type: application/json" -d '{"text": "This movie was really good!", "model": "custom"}'

print("\nTesting with curl (llama model):")
!curl -X POST "$ENDPOINT_URL" -H "Content-Type: application/json" -d '{"text": "Classify the sentiment of this text as positive or negative: This movie was really good! and give sentiment score from 0 to 1 to this sentece", "model": "llama"}'



Testing with curl (custom model):
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK
{"sentiment":"positive","confidence":0.9951962828636169}
Testing with curl (llama model):
Llama Response (raw): I would classify the sentiment of the text as: **Positive**

The phrase "really good" is a strong positive expression, indicating that the speaker enjoyed the movie.

Sentiment score: **0.9** (out of 1)

The score is close to 1, indicating a very positive sentiment, as the speaker uses a strong positive adjective ("really good") to express their opinion about the movie.
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK
{"sentiment":"positive","confidence":0.9}

In [ ]:
# Install ipywidgets if not already installed (uncomment the next line if needed)
!pip install ipywidgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import requests

# Create a text input widget
text_input = widgets.Text(
    value='',
    placeholder='Type your sentence here...',
    description='Input:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Create a dropdown widget for model selection
model_dropdown = widgets.Dropdown(
    options=[('Custom Model', 'custom'), ('Llama 3', 'llama')],
    value='custom',
    description='Model:',
    style={'description_width': 'initial'}
)

# Create a button widget to trigger sentiment analysis
analyze_button = widgets.Button(
    description='Analyze Sentiment',
    button_style='primary',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to analyze sentiment',
    icon='check'
)

# Create an output widget to display results
output = widgets.Output()

# Define the function to call when the button is clicked
def analyze_sentiment(b):
    output.clear_output()  # Clear previous output
    text = text_input.value
    model = model_dropdown.value

    if not text:
        with output:
            print("Please enter some text.")
        return

    # Build the API endpoint URL (replace with your actual endpoint)
    endpoint_url = public_url + "/analyze/"
    payload = {"text": text, "model": model}

    # For the Llama model, modify the prompt as required.
    if model == "llama":
        payload["text"] = ("Classify the sentiment of this text as positive or negative and give final sentiment score [0,1] for this: "
                           + text)

    try:
        response = requests.post(endpoint_url, json=payload)
        # Try to parse the JSON response
        result = response.json()
    except Exception as e:
        result = {"error": str(e)}

    with output:
        print("Result:")
        print(result)

# Attach the function to the button click event
analyze_button.on_click(analyze_sentiment)

# Display the UI components
display(text_input, model_dropdown, analyze_button, output)


Text(value='', description='Input:', layout=Layout(width='400px'), placeholder='Type your sentence here...', s…

Dropdown(description='Model:', options=(('Custom Model', 'custom'), ('Llama 3', 'llama')), style=DescriptionSt…

Button(button_style='primary', description='Analyze Sentiment', icon='check', style=ButtonStyle(), tooltip='Cl…

Output()

INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK
Llama Response (raw): The sentiment of this text is: Positive

The final sentiment score is: 0.9

The text contains the phrase "really good", which is a strong positive expression, indicating a high level of satisfaction or approval. The score of 0.9 reflects this, with 1 being the most positive sentiment possible.
INFO:     34.125.214.192:0 - "POST /analyze/ HTTP/1.1" 200 OK
